# Operations on Inductively Defined Structures


Previously we looked at inductive definitions starting with numbers, lists, binary trees, artihmetic expressions and simple interpretive language. However , beyond defining them, we didn't look at much else. 

This lecture will explore the variety of operations on inductively defined defined structures we have defined so far. 

We will examine two mechanisms for defining these operations: 
    -Using a 'visitor' patter
        - implement the operation as member function as we will see 
        
    - Using pattern matching; this is a special feature of functional languages like scala, lisp, caml, and haskell. You can also find these libraries in python. 
    
    
The first option is generally applicable to most languages. The second option is very special and very powerful. We will focus extensively on the second option of pattern matching, while mentioning what a 'visitor' pattern is. 

## Operations on Numbers

Recall the grammar for the inductively defining numbers 

    NatNum => Z | Succ(NatNum) 

In [30]:
sealed trait NatNum 

case object Z extends NatNum

case class Succ(n: NatNum) extends NatNum 

defined trait NatNum
defined object Z
defined class Succ

The simplest function we can imagine is to add one to a given NaturalNumber. This is easy to implement since this is exactly what Succ does




In [31]:
def addOne(n:NatNum) = Succ(n)

defined function addOne

In [32]:
val two = Succ(Succ(Z))
val three = addOne(two)

two: Succ = Succ(Succ(Z))
three: Succ = Succ(Succ(Succ(Z)))

Now we can write a function minusOne that given a number subtracts one from it. Before we do so, we have to understand how to handle the zero case. We could raise an error/exception saying that it is undefined. This is the best way to do it since it is the most honest 

Heres how it will be implemented 
    minusOne(s) = if s is of the form Succ(t) then return t 
    else s must be Z and return Error 
   
   
To do this we will need a construct that checks if a given input NaturalNumber is of the form Succ(t) and extracts the inner stuff t, but how? 


There are two solutions to this. First is to redfine things to have the minusOne function implemented inide of each class. This solves the problem using the way object oriented programs work. 

In [35]:
sealed trait NatNum1 {
    // all those who inherit from NatNum1 
    // minusOne function, or else ... 
    def minusOne(): NatNum1
}

case object Z1 extends NatNum1{
    // subtracting from zero should throw an error
    def minusOne(): NatNum1 = {
        throw (new IllegalArgumentException("minusOne cannot be called on Zero"))
    }
}

case class Succ1(n: NatNum1) extends NatNum1{
    def minusOne(): NatNum1 = {
        return this.n // returns the inner stuff 
    }
}

defined trait NatNum1
defined object Z1
defined class Succ1

NatNum is the base case class (the constructor looking thing in the sealed trait method). What this means that any class that inherits from it must have all the members that are defined in it. We define a member minusOne() corresponding to the function we wish to implement. Therfore, when we acll the minusOne() funciton on an instance on NatNum, the instance can iether be a 'zero' or 'succ' class. In either case, the object system in scala ensures that the right function gets called. This is an indirect but effective way of finding out the question if the given NatNum is of the form 'zero' or 'Succ' 

In [37]:
val t1 = Z1 
val t2 = Succ1(Succ1(Succ1(Z1)))
val t3 = t2.minusOne()

t1: Z1 = Z1
t2: Succ1 = Succ1(Succ1(Succ1(Z1)))
t3: NatNum1 = Succ1(Succ1(Z1))

looking at the code above from a higher level. 
You have t2 = Succ(Succ(Succ(t1))) so in your head its equal to 3
But the machine sees t2 as just 
    Succ(whatever else is in here)

so when you return this.n you are actually giving it 
    (whatever else is in here) 
    
so when you say val t2 = succ(this stuff) 
    its litterally equal to only 
        val t2 = succ( . . . ) <= the inside stuff doesnt matter
            t2 IS LITTERALLY JUST SUCC(), its just applying the class 

In [38]:
val t4 = t1.minusOne() // this throws an exception which is what we expect 

: 

The second way to do it is using the idea of pattern matching: a very powerful feature that is available in some functional programming languages including scala. Here is how it works. 


An instance object of type NatNum can be of two forms Succ(t) or zero. 
Scala provides a construct similar to the case switch statment in C like 
languages, but much more powerful. 

In [39]:
def minusOne(num: NatNum): NatNum = {
    num match {
        case Succ(t) => t 
        case Z => throw new IllegalArgumentException("minusOne cannot be called on Zero")
    }
}

defined function minusOne

In [40]:
val t5 = minusOne(Succ(Succ(Succ(Z))))

t5: NatNum = Succ(Succ(Z))

In [41]:
val t6 = minusOne(Z)

: 

Let us write code to add two NatNum. The basic idea is: 
    * if the first argument to the call of form zero, then the answer is the second argument since 0 + something = something 
    * if the first argument is of the form Succ(t) then simply make a recursive call to add t with Succ(second argument). We are simply saying
        (1 + t) + n = t + ( 1 + n ) 

In [43]:
def addNatNums(n1: NatNum, n2:NatNum): NatNum = {
    n1 match {
        case Z => n2 
        case Succ(t) => addNatNums(t, Succ(n2))
    }
}

defined function addNatNums

In [48]:
val two = Succ(Succ(Z))
val three = addOne(two)
val five = addNatNums(two,three)
val ten = addNatNums(five,five)

two: Succ = Succ(Succ(Z))
three: Succ = Succ(Succ(Succ(Z)))
five: NatNum = Succ(Succ(Succ(Succ(Succ(Z)))))
ten: NatNum = Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Z))))))))))

for curiosity, how would we implement it using the 'visitor' pattern ? Let us now redefine NatNum1 to require a new member function addNatNums. You can see how instead of pattern matching we simply write the code for the zero case inside of object Z1 and the code for the successor case inside the object Succ1. The idea is the same as before but two cases get split into two different member functions of Z1 and Succ1, respectively. 

In [49]:
sealed trait NatNum1 {
    def minusOne(): NatNum1 
    def addNatNums(n1: NatNum1): NatNum1
}

case object Z1 extends NatNum1{
    def minusOne(): NatNum1 = {
        throw (new IllegalArgumentException("minusOne cannot be called on Zero"))
    }
    
    def addNatNums(n1:NatNum1): NatNum1 = n1 
}

case class Succ1(n: NatNum1) extends NatNum1 {
    def minusOne(): NatNum1 = {
        this.n
    }
    def addNatNums(n1: NatNum1): NatNum1 = this.n.addNatNums(Succ1(n1))
}

defined trait NatNum1
defined object Z1
defined class Succ1

In [52]:
val two = Succ1(Succ1(Z1))
val three = Succ1(two)
val five = two.addNatNums(three)
val ten = five.addNatNums(five)

two: Succ1 = Succ1(Succ1(Z1))
three: Succ1 = Succ1(Succ1(Succ1(Z1)))
five: NatNum1 = Succ1(Succ1(Succ1(Succ1(Succ1(Z1)))))
ten: NatNum1 = Succ1(
  Succ1(Succ1(Succ1(Succ1(Succ1(Succ1(Succ1(Succ1(Succ1(Z1)))))))))
)

Once we have addition, multiplication can now be implemented using recursion, as below. 

In [53]:
def multiplyNatNums(n1: NatNum, n2: NatNum): NatNum = {
    n1 match {
        case Z => { return Z }
        case Succ(t) => {
            // (t + 1) * 2 = t * n2 + n2
            val s1 = multiplyNatNums(t,n2) // t * t2 
            addNatNums(n2,s1) 
        }
    }
}

defined function multiplyNatNums

In [56]:
val four = Succ(Succ(Succ(Succ(Z))))
val five = addOne(four)
val twenty = multiplyNatNums(five,four)

four: Succ = Succ(Succ(Succ(Succ(Z))))
five: Succ = Succ(Succ(Succ(Succ(Succ(Z)))))
twenty: NatNum = Succ(
  Succ(
    Succ(
      Succ(
        Succ(
          Succ(
            Succ(
              Succ(
                Succ(
                  Succ(
                    Succ(
                      Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Z)))))))))
                    )
                  )
                )
              )
            )
          )
        )
      )
    )
  )
)

## Operations on List on Number

Recall that we previously defined a grammar for lists
$$\begin{array}{ccccc}
\textbf{NumList} & \rightarrow & Nil &\ |\  & Cons(\textbf{Num}, \textbf{NumList}) \\
\textbf{Num} & \rightarrow & 0 \ |\ 1\ |\ 2\ |\ 3\ |\ 4\ |\ \cdots \\
\end{array}$$

In [57]:
sealed trait NumList 

case object Nil extends NumList 

case class Cons(hd: Int, t1: NumList) extends NumList 

defined trait NumList
defined object Nil
defined class Cons

There are many exciting things we wish to do lists. The simplest one is to find the length of a list. How do we do that in principle? 
    * the length of the empty list Nil is zero 
    * the length of the list of the form Cons(something,tail) is 1 + length(tail) 
    

Let us use pattern matching to implememt this. 

In [58]:
def listLength(lst: NumList): Int = { 
    lst match{
        case Nil => 0 
        case Cons(_,t1) => 1 + listLength(t1) // _ means anything  
    }
}

defined function listLength

how would we do it using a visitor? Simple, the listLength function is going to become a member of the trait NumList and get implemented in all the classes that inherit from it. 

In [59]:
sealed trait AltNumList{
    def listLength(): Int
}

case object AltNil extends AltNumList{
    def listLength(): Int = {
        0
    }
}

case class AltCons(hd:Int, t1: AltNumList) extends AltNumList{
    def listLength(): Int = {
        1 + t1.listLength()
    }
}

defined trait AltNumList
defined object AltNil
defined class AltCons

In [60]:
val l1 = AltCons(1, AltCons(3, AltCons(7, AltNil)))
val j1 = l1.listLength()

l1: AltCons = AltCons(1, AltCons(3, AltCons(7, AltNil)))
j1: Int = 3

You may now be wondering why we are bothering describing both visitor functions and pattern matches. Aren't they just two different ways of achieveing the same effect? The answer to that is yes for about 90% of the cases, but not always. Pattern matching can make life infinetely more easier. 

I would like to write a function now that does following: 
    given a list, check if it is sorted in ascending order 

In [61]:
def isAscendingOrder(l: NumList): Boolean = 
    l match{
        case Nil => true // an empty list is ascending sure!
        
        // A list with just one element is surely ascending ordered
        case Cons(_, Nil) => true 
        
        case Cons(j1, tl @ Cons(j2, _)) => (j1 <= j2) && isAscendingOrder(tl) 
        // We did something funky:
        // We pattern matched the first two elements of the list to j1 and j2 respectively.
        // Also, we told scala to call Cons(j2, _) by the name tl using the @ symbol. 
        // This is called pattern matching with names.
        
        case _ => {
            assert(false);
            false
        }
        
    }

defined function isAscendingOrder

In [62]:
val b1 = isAscendingOrder( Cons(1, Cons(3, Cons(3, Cons(5, Cons(10, Nil ))))))
val b2 = isAscendingOrder( Cons(5, Cons(3, Cons(3, Cons(5, Cons(10, Nil ))))))
val b3 = isAscendingOrder( Cons(0, Cons(3, Cons(3, Cons(5, Cons(4, Nil ))))))

: 